In [2]:
from src.retrievers.default import DefaultRetriever

retriever = DefaultRetriever()

In [4]:
import os
import dotenv
dotenv.load_dotenv()

azure_configs = {
    "base_url": "https://openai-ai-museum.openai.azure.com/", 
    "embedding_deployment": "text-embedding-3-large",
    "embedding_name": "text-embedding-3-large",
    "model_name": "gpt-4o",
    "api_version": "2024-10-21",  
    "model_deployment": "gpt-4o",
}

In [8]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["model_deployment"],
    model=azure_configs["model_name"],
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_version="2023-05-15",
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["embedding_deployment"],
    model=azure_configs["embedding_name"],
)

In [9]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [10]:
from datasets import Dataset 

data_samples = {
    # user query
    'question': ['Who is Stephy Tsui and what role did she play in the Pictorial Silks exhibition at UMAG?'],
    # generated answer by the model
    'answer': [''],
    # retrievered chunks from the retriever
    'contexts' : [['Pictorialhinese Textiles from the UMAG CSolleictilonksPublished for the exhibition Pictorial Silks: Chinese Textiles from the UMAG Collection at the University Museum and Art Gallery, The University of Hong Kong, 18 September to 6 December, 2020.CuratorKikki LamDesignerStephy TsuiCover illustration: cat.EditionSeptember© University Museum and Art Gallery,The University of Hong Kong, 2020©ISBN978-988-74707-1-7All rights reserved. No part of this publication may be reproduced or transmitted in any']],
    # expected answer
    'ground_truth': ['Stephy Tsui was the designer for the publication associated with the exhibition Pictorial Silks: Chinese Textiles from the UMAG Collection at the University Museum and Art Gallery, The University of Hong Kong.']
}
dataset = Dataset.from_dict(data_samples)

In [11]:
from ragas import evaluate

result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=azure_model, 
    embeddings=azure_embeddings
)

result

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
df = result.to_pandas()
df.head()

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,Who is Stephy Tsui and what role did she play ...,[Pictorialhinese Textiles from the UMAG CSolle...,Stephy Tsui is an artist.,Stephy Tsui was the designer for the publicati...,1.0,0.0,0.656221,1.0
